# pyResToolbox - Documented Examples

This notebook contains every documented code example from the pyResToolbox API reference, organized by module. Each cell is self-contained and demonstrates the usage of a specific function.

---
## 1. Gas PVT & Flow

Gas property calculations supporting hydrocarbon + impurity mixtures (CO2, H2S, N2, H2).

### Calculation Methods
- **zmethod**: `'DAK'` (default), `'HY'`, `'WYW'`, `'BUR'`
- **cmethod**: `'PMC'` (default), `'SUT'`, `'BUR'`

In [ ]:
import numpy as np
from pyrestoolbox import gas

### gas_z - Gas Z-Factor

Calculating gas Z-Factor of pure methane using DAK and PMC for critical properties

In [ ]:
gas.gas_z(p=2350, sg=0.68, degf=180, zmethod='DAK', cmethod='PMC')

Calculating gas Z-Factor of pure CO2

In [ ]:
gas.gas_z(p=2350, sg=0.68, degf=180, co2=1.0, zmethod='BUR', cmethod='BUR')

Calculating gas SG, and then gas Z-Factor of a mixture of 5% CO2, 10% H2S, 0% N2 and 20% H2 (remainder natural gas with MW = 19)

In [ ]:
gsg = gas.gas_sg(hc_mw=19.0, co2=0.05, h2s=0.10, n2=0, h2=0.20)
print('Gas SG:', gsg)
gas.gas_z(p=2350, sg=gsg, degf=180, co2=0.05, h2s=0.10, n2=0, h2=0.20, zmethod='BUR', cmethod='BUR')

Using class objects instead of string method selection

In [ ]:
gas.gas_z(p=2350, sg=0.68, degf=180, zmethod=gas.z_method.DAK, cmethod=gas.c_method.PMC)

### gas_sg - Mixture Gas Specific Gravity

Calculate SG of a gas mixture containing impurities. Essential for BNS/BUR method inputs.

In [ ]:
# Pure hydrocarbon gas (MW = 19)
print('Pure HC gas SG:', gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0))

# 20% H2 blend with natural gas
print('20% H2 blend SG:', gas.gas_sg(hc_mw=19.0, co2=0.05, h2s=0.10, n2=0, h2=0.20))

# Pure CO2
print('Pure CO2 SG:', gas.gas_sg(hc_mw=19.0, co2=1.0, h2s=0, n2=0, h2=0))

# High H2 blend (50% H2)
print('50% H2 blend SG:', gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.50))

### gas_tc_pc - Gas Critical Properties

In [ ]:
gas.gas_tc_pc(sg=0.7, co2=0.15)

In [ ]:
# With user-specified Tc and SUT method
gas.gas_tc_pc(sg=0.7, co2=0.15, tc=365, cmethod='SUT')

In [ ]:
# BUR critical properties for a gas with 10% CO2 and 5% H2S
gas.gas_tc_pc(sg=0.75, co2=0.10, h2s=0.05, cmethod='BUR')

In [ ]:
# BUR auto-selects when H2 is present
gas.gas_tc_pc(sg=0.6, h2=0.30)

### gas_z - Z-Factor with various methods and impurities

In [ ]:
# DAK method (default) with N2 and CO2
gas.gas_z(p=1000, sg=0.75, degf=160, n2=0.02, co2=0.17)

In [ ]:
# HY method
gas.gas_z(p=1000, sg=0.75, degf=160, n2=0.02, co2=0.17, zmethod='HY')

In [ ]:
# Array of pressures with SUT critical property method
gas.gas_z(p=[1000, 2000], sg=0.75, degf=160, cmethod='SUT', n2=0.02, co2=0.17)

BUR/BNS Z-Factor across a pressure range for different gas compositions. The BNS model uses a tuned 5-component Peng Robinson EOS that natively handles CO2, H2S, N2, and H2 at any concentration up to pure inerts.

In [ ]:
import matplotlib.pyplot as plt

# Compare Z-factor across pressures for different gas compositions using BUR
pressures = list(range(500, 10001, 500))

# Pure methane (BUR)
z_ch4 = gas.gas_z(p=pressures, sg=0.5539, degf=200, zmethod='BUR', cmethod='BUR')

# 15% CO2 blend
z_co2 = gas.gas_z(p=pressures, sg=0.7, degf=200, co2=0.15, zmethod='BUR', cmethod='BUR')

# 20% H2 blend (auto-selects BUR)
sg_h2 = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
z_h2 = gas.gas_z(p=pressures, sg=sg_h2, degf=200, h2=0.20, zmethod='BUR', cmethod='BUR')

# Pure CO2
z_pureco2 = gas.gas_z(p=pressures, sg=1.5196, degf=200, co2=1.0, zmethod='BUR', cmethod='BUR')

plt.plot(pressures, z_ch4, label='Pure CH4')
plt.plot(pressures, z_co2, label='15% CO2 blend')
plt.plot(pressures, z_h2, label='20% H2 blend')
plt.plot(pressures, z_pureco2, label='Pure CO2')
plt.xlabel('Pressure (psia)')
plt.ylabel('Z-Factor')
plt.title('BNS Z-Factor: CO2 and H2 Blends')
plt.legend()
plt.grid(True)
plt.show()

### gas_ug - Gas Viscosity

In [ ]:
gas.gas_ug(p=1000, sg=0.75, degf=180, zmethod='HY', cmethod='SUT')

In [ ]:
gas.gas_ug(p=1000, sg=0.75, degf=180)

BUR viscosity uses a tuned LBC (Lohrenz-Bray-Clark) method instead of the default Lee-Gonzalez-Eakin, enabling viscosity calculations for CO2-rich and H2-rich gases.

In [ ]:
# BUR/LBC viscosity for pure CO2
print('Pure CO2 viscosity:', gas.gas_ug(p=2000, sg=1.5196, degf=200, co2=1.0, zmethod='BUR', cmethod='BUR'))

# Viscosity of a 20% H2 blend (BUR auto-selects)
sg_h2mix = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
print('20% H2 blend viscosity:', gas.gas_ug(p=2000, sg=sg_h2mix, degf=200, h2=0.20))

# Viscosity comparison across pressure range for conventional vs H2 blend
pressures = list(range(500, 8001, 500))
ug_conv = gas.gas_ug(p=pressures, sg=0.7, degf=200)
ug_h2 = gas.gas_ug(p=pressures, sg=sg_h2mix, degf=200, h2=0.20, zmethod='BUR', cmethod='BUR')
ug_co2 = gas.gas_ug(p=pressures, sg=1.5196, degf=200, co2=1.0, zmethod='BUR', cmethod='BUR')

plt.plot(pressures, ug_conv, label='Conventional gas (LEG)')
plt.plot(pressures, ug_h2, label='20% H2 blend (LBC)')
plt.plot(pressures, ug_co2, label='Pure CO2 (LBC)')
plt.xlabel('Pressure (psia)')
plt.ylabel('Viscosity (cP)')
plt.title('Gas Viscosity: Conventional vs BNS/LBC Method')
plt.legend()
plt.grid(True)
plt.show()

### gas_cg - Gas Compressibility

In [ ]:
gas.gas_cg(p=2000, sg=0.68, degf=120, co2=0.05)

In [ ]:
# Array of pressures
gas.gas_cg(p=np.array([1000, 2000]), sg=0.68, degf=120, co2=0.05)

In [ ]:
# Gas compressibility for pure CO2 and H2 blend using BUR
print('Cg pure CO2:', gas.gas_cg(p=2000, sg=1.5196, degf=200, co2=1.0, zmethod='BUR', cmethod='BUR'))

sg_h2 = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
print('Cg 20% H2 blend:', gas.gas_cg(p=2000, sg=sg_h2, degf=200, h2=0.20, zmethod='BUR', cmethod='BUR'))

### gas_bg - Gas Formation Volume Factor

In [ ]:
gas.gas_bg(p=3000, sg=0.78, degf=240)

In [ ]:
# Inverse Bg (expansion factor) for array of pressures
1 / gas.gas_bg(p=[3000, 5000], sg=0.78, degf=240)

In [ ]:
# Bg for pure CO2 using BUR
print('Bg pure CO2:', gas.gas_bg(p=3000, sg=1.5196, degf=200, co2=1.0, zmethod='BUR', cmethod='BUR'))

# Bg for 20% H2 blend
sg_h2 = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
print('Bg 20% H2 blend:', gas.gas_bg(p=3000, sg=sg_h2, degf=200, h2=0.20, zmethod='BUR', cmethod='BUR'))

### gas_den - Gas Density

In [ ]:
gas.gas_den(p=2000, sg=0.75, degf=150, zmethod='HY', cmethod='SUT', n2=0.02, co2=0.15, h2s=0.02)

In [ ]:
# Gas density for pure CO2 using BUR
print('Pure CO2 density:', gas.gas_den(p=2000, sg=1.5196, degf=200, co2=1.0, zmethod='BUR', cmethod='BUR'))

# Gas density for 20% H2 blend - much lighter than conventional gas
sg_h2 = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
print('20% H2 blend density:', gas.gas_den(p=2000, sg=sg_h2, degf=200, h2=0.20, zmethod='BUR', cmethod='BUR'))

# Conventional gas density for comparison
print('Conventional gas density:', gas.gas_den(p=2000, sg=0.75, degf=200))

### gas_water_content - Saturated Water Vapor in Gas

In [ ]:
gas.gas_water_content(p=1500, degf=165)

### gas_ponz2p - Convert P/Z to Pressure

In [ ]:
gas.gas_ponz2p(poverz=2500, sg=0.75, degf=165)

In [ ]:
gas.gas_ponz2p(poverz=[2500, 5000], sg=0.75, degf=165)

### gas_grad2sg - Invert Gas Gradient to SG

In [ ]:
gas.gas_grad2sg(grad=0.0657, p=2500, degf=175)

### gas_dmp - Delta Pseudopressure

In [ ]:
# Positive result when p1 < p2
gas.gas_dmp(p1=1000, p2=2000, degf=185, sg=0.78, zmethod='HY', cmethod='SUT', n2=0.05, co2=0.1, h2s=0.02)

In [ ]:
# Negative result when p1 > p2, using user-specified Tc/Pc
gas.gas_dmp(p1=2000, p2=1000, degf=185, sg=0.78, tc=371, pc=682)

In [ ]:
# Pseudopressure for pure CO2 using BUR
print('dmp pure CO2:', gas.gas_dmp(p1=1000, p2=3000, degf=200, sg=1.5196, co2=1.0, zmethod='BUR', cmethod='BUR'))

# Pseudopressure for 20% H2 blend (BUR auto-selects)
sg_h2 = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
print('dmp 20% H2 blend:', gas.gas_dmp(p1=1000, p2=3000, degf=200, sg=sg_h2, h2=0.20))

### gas_fws_sg - Full Wellstream Gas SG

In [ ]:
gas.gas_fws_sg(sg_g=0.855, cgr=30, api_st=53)

### gas_rate_radial - Gas Radial Flow Rate

In [ ]:
gas.gas_rate_radial(k=5, h=50, pr=2000, pwf=750, r_w=0.3, r_ext=1500, degf=180, sg=0.75, D=0.01, S=5)

In [ ]:
# Array of reservoir pressures
gas.gas_rate_radial(k=1, h=50, pr=[2000, 1000], pwf=750, r_w=0.3, r_ext=1500, degf=180, sg=0.75, D=0.01, S=5)

In [ ]:
# Radial gas flow rate for pure CO2 using BUR
print('Pure CO2 rate:', gas.gas_rate_radial(
    k=5, h=50, pr=2000, pwf=750, r_w=0.3, r_ext=1500, degf=180,
    sg=1.5196, co2=1.0, zmethod='BUR', cmethod='BUR', D=0.01, S=5
))

# Radial gas flow rate for 20% H2 blend
sg_h2 = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
print('20% H2 blend rate:', gas.gas_rate_radial(
    k=5, h=50, pr=2000, pwf=750, r_w=0.3, r_ext=1500, degf=180,
    sg=sg_h2, h2=0.20, zmethod='BUR', cmethod='BUR', D=0.01, S=5
))

### gas_rate_linear - Gas Linear Flow Rate

In [ ]:
gas.gas_rate_linear(k=0.1, area=50, length=200, pr=2000, pwf=250, degf=180, sg=0.8)

In [ ]:
# Array of reservoir pressures
gas.gas_rate_linear(k=0.1, area=50, length=200, pr=[2000, 1000, 500], pwf=250, degf=180, sg=0.8)

In [ ]:
# Linear gas flow rate for pure CO2 using BUR
print('Pure CO2 linear rate:', gas.gas_rate_linear(
    k=0.1, area=50, length=200, pr=2000, pwf=250, degf=180,
    sg=1.5196, co2=1.0, zmethod='BUR', cmethod='BUR'
))

# Linear gas flow rate for 20% H2 blend
sg_h2 = gas.gas_sg(hc_mw=19.0, co2=0, h2s=0, n2=0, h2=0.20)
print('20% H2 blend linear rate:', gas.gas_rate_linear(
    k=0.1, area=50, length=200, pr=2000, pwf=250, degf=180,
    sg=sg_h2, h2=0.20, zmethod='BUR', cmethod='BUR'
))

---
## 2. Oil PVT & Flow Rates

Oil property calculations including bubble point, solution GOR, FVF, density, viscosity, and compressibility.

### Calculation Methods
- **pbmethod**: `'VALMC'` (default), `'STAN'`, `'VELAR'`
- **rsmethod**: `'VELAR'` (default), `'STAN'`, `'VALMC'`
- **bomethod**: `'MCAIN'` (default), `'STAN'`
- **denomethod**: `'SWMH'` (default)
- **comethod**: `'EXPLT'` (default)

In [ ]:
from pyrestoolbox import oil

### oil_pbub - Bubble Point Pressure

Methods can be specified via string or class object

In [ ]:
# Via string
oil.oil_pbub(api=43, degf=185, rsb=2350, sg_g=0.72, pbmethod='STAN')

In [ ]:
# Via class object
oil.oil_pbub(api=43, degf=185, rsb=2350, sg_g=0.72, pbmethod=oil.pb_method.STAN)

### oil_ja_sg - Oil SG from Jacoby Aromaticity

In [ ]:
oil.oil_ja_sg(mw=150, ja=0.5)

### oil_twu_props - Twu Critical Properties

Returns (sg, tb_R, tc_R, pc_psia, vc_ft3_per_lbmol)

In [ ]:
oil.oil_twu_props(mw=225, ja=0.5)

### oil_rs_st - Incremental GOR post Separation

In [ ]:
oil.oil_rs_st(psp=114.7, degf_sp=80, api=38)

### oil_pbub - Bubble Point Pressure

In [ ]:
# Default VALMC method using sg_g
oil.oil_pbub(api=43, degf=185, rsb=2350, sg_g=0.72)

In [ ]:
# Standing method using sg_sp
oil.oil_pbub(api=43, degf=185, rsb=2350, sg_sp=0.72, pbmethod='STAN')

### oil_rs_bub - Solution GOR at Bubble Point

In [ ]:
oil.oil_rs_bub(api=43, degf=185, pb=5179.5, sg_sp=0.72)

### oil_rs - Solution GOR at Pressure

In [ ]:
# With both pb and rsb specified
oil.oil_rs(api=43, degf=185, sg_sp=0.72, p=3000, pb=5179.5, rsb=2370)

In [ ]:
# With rsb only (pb calculated from correlation)
oil.oil_rs(api=43, degf=185, sg_sp=0.72, p=3000, rsb=2370)

In [ ]:
# With pb only (rsb calculated from correlation)
oil.oil_rs(api=43, degf=185, sg_sp=0.72, p=3000, pb=5180)

In [ ]:
# Using Standing method
oil.oil_rs(api=43, degf=185, sg_sp=0.72, p=3000, pb=5180, rsmethod='STAN')

### oil_co - Oil Compressibility

In [ ]:
# Above bubble point (undersaturated)
oil.oil_co(p=4500, api=47, degf=180, sg_sp=0.72, rsb=2750)

In [ ]:
# Below bubble point (saturated)
oil.oil_co(p=2000, api=47, degf=180, sg_sp=0.72, rsb=2750, pb=4945)

### oil_deno - Live Oil Density

In [ ]:
oil.oil_deno(p=2000, degf=165, rs=1000, rsb=2000, sg_g=0.72, api=38)

### oil_bo - Oil Formation Volume Factor

In [ ]:
# McCain method (default, density-based)
oil.oil_bo(p=2000, pb=3000, degf=165, rs=1000, rsb=2000, sg_o=0.8, sg_g=0.68)

In [ ]:
# Standing method
oil.oil_bo(p=2000, pb=3000, degf=165, rs=1000, rsb=2000, sg_o=0.8, sg_g=0.68, bomethod='STAN')

### oil_viso - Oil Viscosity

In [ ]:
oil.oil_viso(p=2000, api=38, degf=165, pb=3500, rs=1000)

### make_bot_og - Black Oil Table Generation

In [ ]:
results = oil.make_bot_og(pvto=False, pi=4000, api=38, degf=175, sg_g=0.68, pmax=5500, pb=4500, nrows=10, export=False)
df = results['bot']
st_deno = results['deno']
st_deng = results['deng']
res_denw = results['denw']
res_cw = results['cw']
visw = results['uw']
pb = results['pb']
rsb = results['rsb']
rsb_frac = results['rsb_scale']
usat = results['usat']

print(f'Pb: {pb} psia')
print(f'ST Oil Density: {st_deno:.2f} lb/cuft')
print(f'ST Gas Density: {st_deng:.4f} lb/cuft')
print()
df

### sg_evolved_gas - SG of Evolved Gas from Oil

In [ ]:
oil.sg_evolved_gas(p=2000, degf=185, rsb=2370, api=43, sg_sp=0.72)

### sg_st_gas - SG of Gas post Separator

In [ ]:
oil.sg_st_gas(114.7, rsp=1500, api=42, sg_sp=0.72, degf_sp=80)

### sgg_wt_avg - Weighted Average Surface Gas SG

In [ ]:
oil.sgg_wt_avg(sg_sp=0.72, rsp=1000, sg_st=1.1, rst=5)

### oil_api / oil_sg - API-SG Conversions

In [ ]:
oil.oil_api(sg_value=0.82)

In [ ]:
oil.oil_sg(api_value=45)

### oil_rate_radial - Oil Radial Flow Rate

In [ ]:
# Scalar with Vogel correction
oil.oil_rate_radial(k=20, h=20, pr=1500, pwf=250, r_w=0.3, r_ext=1500, uo=0.8, bo=1.4, vogel=True, pb=1800)

In [ ]:
# Array of reservoir pressures
oil.oil_rate_radial(k=20, h=20, pr=[1500, 2000], pwf=250, r_w=0.3, r_ext=1500, uo=0.8, bo=1.4, vogel=True, pb=1800)

### oil_rate_linear - Oil Linear Flow Rate

In [ ]:
oil.oil_rate_linear(k=0.1, area=15000, pr=3000, pwf=500, length=500, uo=0.4, bo=1.5)

In [ ]:
# Array of permeabilities
oil.oil_rate_linear(k=[0.1, 1, 5, 10], area=15000, pr=3000, pwf=500, length=500, uo=0.4, bo=1.5)

---
## 3. Brine PVT

Brine properties with differing degrees of methane or CO2 saturation.

In [ ]:
from pyrestoolbox import brine

### brine_props - Methane-saturated Brine Properties

Returns (Bw, Density_sg, Viscosity_cP, Compressibility_1/psi, Rsw_scf/stb)

In [ ]:
bw, lsg, visw, cw, rsw = brine.brine_props(p=160, degf=135, wt=1.5, ch4_sat=1.0)
print('Bw:', bw)
print('SGw:', lsg)
print('Visw:', visw)
print('Cw:', cw)
print('Rsw:', rsw)

### CO2_Brine_Mixture - CO2-saturated Brine Properties

Usage example for 5000 psia x 275 deg F and 3% NaCl brine (field units)

In [ ]:
mix = brine.CO2_Brine_Mixture(pres=5000, temp=275, ppm=30000, metric=False)
print('Bw [CO2 Sat, Pure, Fresh]:', mix.bw)
print('x [xCO2, xH2O]:', mix.x)
print('y [yCO2, yH2O]:', mix.y)
print('Brine Density [CO2 Sat, Pure, Fresh]:', mix.bDen)
print('Brine Viscosity [CO2 Sat, Pure, Fresh]:', mix.bVis)
print('Rs (scf/stb):', mix.Rs)

Usage example for 175 Bara x 85 degC and 0% NaCl brine (metric units)

In [ ]:
mix = brine.CO2_Brine_Mixture(pres=175, temp=85)
print('Rs (sm3/sm3):', mix.Rs)

### make_pvtw_table - PVTW Table Generation

In [ ]:
result = brine.make_pvtw_table(pi=3000, degf=200, wt=0, ch4_sat=0)
print('Reference Bw:', result['bw_ref'])
print('Reference Cw:', result['cw_ref'])
print('Reference Visw:', result['visw_ref'])
print()
result['table'].head()

---
## 4. Permeability Layering

Functions to characterize heterogeneity via the Lorenz coefficient and generate permeability distributions.

Two methods available:
- **'EXP'** (default): Exponential formulation
- **'LANG'**: Langmuir formulation

In [ ]:
from pyrestoolbox import layer

### lorenz2b - Lorenz Coefficient to Beta

In [ ]:
# Langmuir method
layer.lorenz2b(0.75, lrnz_method='LANG')

In [ ]:
# Exponential method (default)
layer.lorenz2b(0.75)

### lorenzfromb - Beta to Lorenz Coefficient

In [ ]:
# Langmuir method
layer.lorenzfromb(16.139518537603912, lrnz_method='LANG')

In [ ]:
# Exponential method (default)
layer.lorenzfromb(7.978108090962671)

### lorenz_from_flow_fraction - Lorenz from Observed Flow/Thickness Fractions

60% of the observed flow comes from 15% of the net thickness

In [ ]:
lorenz_factor = layer.lorenz_from_flow_fraction(kh_frac=0.6, phih_frac=0.15)
lorenz_factor

### lorenz_2_flow_frac - Expected Flow Fraction from Lorenz

In [ ]:
layer.lorenz_2_flow_frac(lorenz=0.6759312029093838, phih_frac=0.15)

### lorenz_2_layers - Generate Permeability Distribution

In [ ]:
# 5 equal-thickness layers with shuffle
layer.lorenz_2_layers(lorenz=0.67, nlayers=5, k_avg=10, shuffle=True)

In [ ]:
# Specified phi_h fractions
layer.lorenz_2_layers(lorenz=0.67, k_avg=10, phi_h_fracs=[0.05, 0.5])

---
## 5. Simulation Helpers

Simulation-oriented tools including relative permeability tables, aquifer influence functions, and flash calculations.

In [ ]:
import matplotlib.pyplot as plt
from pyrestoolbox import simtools

### influence_tables - Van Everdingen-Hurst Aquifer Influence

In [ ]:
ReDs = [1.5, 2, 3, 5, 10, 25, 1000]
tds, pds = simtools.influence_tables(ReDs=ReDs, export=False)

for p, pd in enumerate(pds):
    plt.plot(tds, pd, label=str(ReDs[p]))

plt.xscale('log')
plt.yscale('log')
plt.legend(loc='upper left')
plt.grid(which='both')
plt.xlabel('Dimensionless Time (tD)')
plt.ylabel('Dimensionless Pressure Drop (PD)')
plt.title('Constant Terminal Rate Solution')
plt.show()

### rr_solver - Rachford-Rice Flash Calculation

Returns (N_iterations, yi_vapor, xi_liquid, V_fraction, L_fraction)

In [ ]:
n_it, yi, xi, V, L = simtools.rr_solver(
    zi=np.array([0.7, 0.15, 0.1, 0.05]),
    ki=np.array([50, 5, 0.5, 0.01])
)
print(f'Iterations: {n_it}')
print(f'Vapor compositions (yi): {yi}')
print(f'Liquid compositions (xi): {xi}')
print(f'Vapor fraction (V): {V}')
print(f'Liquid fraction (L): {L}')

### rel_perm_table - Relative Permeability Tables

SGOF with LET curves

In [ ]:
df = simtools.rel_perm_table(
    rows=25, krtable='SGOF', krfamily='LET',
    kromax=1, krgmax=1, swc=0.2, sorg=0.15,
    Lo=2.5, Eo=1.25, To=1.75, Lg=1.2, Eg=1.5, Tg=2.0
)
plt.plot(df['Sg'], df['Krgo'], c='r', label='Gas')
plt.plot(df['Sg'], df['Krog'], c='g', label='Oil')
plt.title('SGOF Gas Oil LET Relative Permeability Curves')
plt.xlabel('Sg')
plt.ylabel('Kr')
plt.legend()
plt.grid('both')
plt.show()

SWOF with Corey curves

In [ ]:
df = simtools.rel_perm_table(
    rows=25, krtable='SWOF',
    kromax=1, krwmax=0.25, swc=0.15, swcr=0.2, sorw=0.15,
    no=2.5, nw=1.5
)
plt.plot(df['Sw'], df['Krow'], c='g', label='Oil')
plt.plot(df['Sw'], df['Krwo'], c='b', label='Water')
plt.title('SWOF Water Oil Corey Relative Permeability Curves')
plt.xlabel('Sw')
plt.ylabel('Kr')
plt.legend()
plt.grid('both')
plt.show()

---
## 6. Component Critical Properties Library

Excel-based component property database with EOS-model-specific parameters (PR79, PR77, SRK, RK).

In [ ]:
from pyrestoolbox import library

### prop - Return Critical Property

In [ ]:
library.prop(comp='CH4', prop='Pc_psia')

In [ ]:
# Compare VTran for PR79 vs SRK models
library.prop(comp='C3', prop='VTran'), library.prop(comp='C3', prop='VTran', model='SRK')

### components - List of Available Components

In [ ]:
print(library.components)

### names - Long-form Component Names

In [ ]:
print(library.names)

### property_list - Available Properties

In [ ]:
print(library.property_list)

### models - Available EOS Models

In [ ]:
print(library.models)

### df - Full Library DataFrame

In [ ]:
library.df